<a href="https://www.kaggle.com/code/luminlab/ber-curation?scriptVersionId=184056641" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://www.kaggle.com/code/luminlab/ber-curation?scriptVersionId=183010840" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://www.kaggle.com/code/luminlab/ber-curation?scriptVersionId=182786979" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://www.kaggle.com/code/luminlab/ber-curation?scriptVersionId=182712552" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://www.kaggle.com/code/luminlab/ber-curation?scriptVersionId=182257992" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Details of Steps can be found on Google Slides via:¶

https://docs.google.com/presentation/d/1sb3QkXiYooHqi3p-tkGVUqwqFKd-601_pzU96W1drw0/edit?usp=sharing

In [ ]:
from kaggle_secrets import UserSecretsClient
wandb_key_label = "WANDB_KEY"
wandb_key= UserSecretsClient().get_secret(wandb_key_label)

In [ ]:
import git
git.Repo.clone_from('https://github.com/Lumin-Lab/BerCuration', '/kaggle/working/scarf')

In [ ]:
!pip install -r /kaggle/working/scarf/requirements.txt

In [ ]:
import os
def save_csv_file(df, path):
    dir_name = os.path.dirname(path)
    if dir_name:
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
    df.to_csv(path, index=False)

In [ ]:
raw_data_path = "/kaggle/input/ber-stratified-samples/BER_stratified_sample.csv"
n_splits = 5
output_dir = "/kaggle/working/output"
config_dir="/kaggle/working/scarf/configs"
scarf_model_name = "scarf"
mlp_model_name = "mlp"
scarf_batch_size = 32
# scarf_epochs = 25 
scarf_epochs = 1 
scarf_lr = 3e-5
scarf_emb_dim = 32
scarf_encoder_depth = 3
scarf_corruption_rate=0.3
# max_depth = 6
max_depth = 2

In [ ]:
import pandas as pd
X_raw = pd.read_csv(raw_data_path)

In [ ]:
from sklearn.model_selection import KFold
import os
kf = KFold(n_splits=n_splits, random_state=42, shuffle=True)
for i, (train_index, test_index) in enumerate(kf.split(X_raw)):
    save_csv_file(X_raw.iloc[train_index], f"{output_dir}/split_{i+1}/raw_train.csv")
    save_csv_file(X_raw.iloc[test_index], f"{output_dir}/split_{i+1}/raw_test.csv")

In [ ]:
"""input: data_path={raw_data_path}" ,
output:{output_dir}/processed.csv """

command = f"""
    python /kaggle/working/scarf/get_processed_dataset.py \
      --config_dir={config_dir} \
      --output_dir="{output_dir}" \
      --data_path={raw_data_path} \
      --output_csv_name "processed" \
      --is_train
    """
os.system(command)

In [ ]:
files_to_copy = ["column_type_classification.yaml", "encoder.joblib", "scaler.joblib", "train_stats.json"]
for i in range(n_splits):
    for file_name in files_to_copy:
        src = f"{output_dir}/{file_name}"
        dest = f"{output_dir}/split_{i+1}/{file_name}"
        os.system(f"cp {src} {dest}")

In [ ]:
for i in range(n_splits):
    """input: data_path "{output_dir}/split_{i+1}/raw_train.csv" ,
    output:{output_dir}/split_{i+1}/processed_train.csv """
    command = f"""
    python /kaggle/working/scarf/get_processed_dataset.py \
      --config_dir "{config_dir}" \
      --output_dir "{output_dir}/split_{i+1}" \
      --data_path "{output_dir}/split_{i+1}/raw_train.csv" \
      --output_csv_name "processed_train" \
    """
    os.system(command)
    """input: data_path "{output_dir}/split_{i+1}/raw_test.csv" ,
    output:{output_dir}/split_{i+1}/processed_test.csv """
    command = f"""
    python /kaggle/working/scarf/get_processed_dataset.py \
      --config_dir "{config_dir}" \
      --output_dir "{output_dir}/split_{i+1}" \
      --data_path "{output_dir}/split_{i+1}/raw_test.csv" \
      --output_csv_name "processed_test"
    """
    os.system(command)

**Train the SCARF Encoder on the Train Dataset**

In [ ]:
import wandb
wandb.login(key=wandb_key)

In [ ]:
import os
"""
Input: train_data_path="{output_dir}/split_{i+1}/processed_train.csv"
 
Output: /kaggle/working/output/split_{split}/scarf.pt  
"""
for i in range(n_splits):
    command = f"""
    python /kaggle/working/scarf/run_scarf.py\
      --config_dir={config_dir} \
      --output_dir="{output_dir}/split_{i+1}" \
      --train_data_path="{output_dir}/split_{i+1}/processed_train.csv"\
      --batch_size={scarf_batch_size} \
      --epochs={scarf_epochs} \
      --lr={scarf_lr} \
      --emb_dim={scarf_emb_dim} \
      --encoder_depth={scarf_encoder_depth} \
      --model_name="{scarf_model_name}" \
      --corruption_rate={scarf_corruption_rate} \
      --wandb_project_name='SCARF_Project' \
      --wandb_entity='urbancomp' 
    """

    os.system(command)

**Obtain the SCARF embeddings for different train splits, and save the result**

In [ ]:
"""
Input: data_path="{output_dir}/split_{i+1}/processed_train.csv"
 
Output: {output_dir}/split_{i+1}/train.npy
"""
for i in range(n_splits):
    command = f"""
    python /kaggle/working/scarf/get_scarf_embedding.py \
      --config_dir={config_dir} \
      --output_dir="{output_dir}/split_{i+1}" \
      --data_path="{output_dir}/split_{i+1}/processed_train.csv" \
      --batch_size={scarf_batch_size} \
      --epochs={scarf_epochs} \
      --lr={scarf_lr} \
      --emb_dim={scarf_emb_dim} \
      --encoder_depth={scarf_encoder_depth} \
      --model_name={scarf_model_name} \
      --corruption_rate={scarf_corruption_rate} \
      --embedding_save_name="train"
    """

    os.system(command)

**Obtain the SCARF embeddings for different test splits, and save the result**

In [ ]:
"""
Input: data_path="{output_dir}/split_{i+1}/processed_test.csv"
 
Output: {output_dir}/split_{i+1}/test.npy
"""
for i in range(n_splits):
    command = f"""
    python /kaggle/working/scarf/get_scarf_embedding.py\
      --config_dir={config_dir} \
      --output_dir="{output_dir}/split_{i+1}" \
      --data_path="{output_dir}/split_{i+1}/processed_test.csv" \
      --batch_size={scarf_batch_size} \
      --epochs={scarf_epochs} \
      --lr={scarf_lr} \
      --emb_dim={scarf_emb_dim} \
      --encoder_depth={scarf_encoder_depth} \
      --model_name={scarf_model_name} \
      --corruption_rate={scarf_corruption_rate} \
      --embedding_save_name="test"
    """

    os.system(command)

In [ ]:
"""
Input: data_path="{output_dir}/processedt.csv"
 
Output: {output_dir}/split_1/all_emb.npy
"""

command = f"""
    python /kaggle/working/scarf/get_scarf_embedding.py \
      --config_dir={config_dir} \
      --output_dir="{output_dir}/split_1" \
      --data_path="{output_dir}/processed.csv" \
      --batch_size={scarf_batch_size} \
      --epochs={scarf_epochs} \
      --lr={scarf_lr} \
      --emb_dim={scarf_emb_dim} \
      --encoder_depth={scarf_encoder_depth} \
      --model_name={scarf_model_name} \
      --corruption_rate={scarf_corruption_rate} \
      --embedding_save_name="all_emb"
    """

os.system(command)

**Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
def get_rf(max_depth=max_depth):
    return RandomForestClassifier(max_depth=max_depth, random_state=42)

**Original Data**

In [ ]:
import numpy as np
processed = pd.read_csv(f"{output_dir}/processed.csv")
all_emb = np.load(f"{output_dir}/split_1/all_emb.npy")

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import cross_val_predict
from cleanlab import Datalab
from sklearn.metrics import f1_score

In [ ]:
X_processed = processed.drop(columns=["EnergyRating"])
labels = processed['EnergyRating']
clf = get_rf()
pred_probs = cross_val_predict(
    clf,
    X_processed,
    labels,
    cv=kf,
    method="predict_proba",
)
KNN = NearestNeighbors(metric='euclidean')
KNN.fit(X_processed.values)

knn_graph = KNN.kneighbors_graph(mode="distance")
data = {"X": X_processed.values, "y": labels}

lab = Datalab(data, label_name="y")
lab.find_issues(pred_probs=pred_probs, knn_graph=knn_graph)

display(lab.report())


In [ ]:
# Label Issues
label_issues_num =  54404
outlier_nums = 2296
duplicates_nums = 8785

issue_results = lab.get_issues("label")
sorted_issues = issue_results.sort_values("label_score").index

sorted_issues_df = pd.DataFrame({
    "Unnamed: 0": X_raw.loc[sorted_issues, "Unnamed: 0"]
})
save_csv_file(sorted_issues_df[:label_issues_num], f"{output_dir}/random_forest/data/label_issues.csv")
print(len(sorted_issues_df))

print("Label Issues")
display(X_raw.iloc[sorted_issues].assign(
    given_label=labels.iloc[sorted_issues],
    predicted_label=issue_results["predicted_label"].iloc[sorted_issues]
).head())

# Outliers
outlier_results = lab.get_issues("outlier")
sorted_outliers= outlier_results.sort_values("outlier_score").index
print("Outliers")
display(X_raw.iloc[sorted_outliers].head())

sorted_outliers_df = pd.DataFrame({
    "Unnamed: 0": X_raw.loc[sorted_outliers[:outlier_nums], "Unnamed: 0"]
})
print(len(sorted_outliers_df))
save_csv_file(sorted_outliers_df, f"{output_dir}/random_forest/data/outliers.csv")


# Near-duplicate issues
duplicate_results = lab.get_issues("near_duplicate")
sorted_duplicates = duplicate_results.sort_values("near_duplicate_score").index
print("Near-duplicate issues")
display(duplicate_results.sort_values("near_duplicate_score").head())
sorted_duplicates_df = pd.DataFrame({
    "Unnamed: 0": X_raw.loc[sorted_duplicates, "Unnamed: 0"]
})
print(len(sorted_duplicates_df))
save_csv_file(sorted_duplicates_df[:duplicates_nums], f"{output_dir}/random_forest/data/duplicates.csv")


**Before Removal**

In [ ]:
results = []

for i in range(n_splits):
    clf = get_rf()
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_train.csv")
    test_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_test.csv")
    clf.fit(train_df.drop(columns=["EnergyRating"]), train_df["EnergyRating"])
    acc = clf.score(test_df.drop(columns=["EnergyRating"]), test_df["EnergyRating"])
    predict = clf.predict(test_df.drop(columns=["EnergyRating"]))
    f1 = f1_score(test_df["EnergyRating"], predict, average='macro')
    print(f"Split: {i+1}, Accuracy: {acc}, F1 Score: {f1}")
    results.append({"Split": i+1, "Accuracy": acc, "F1 Score": f1})

# Save the results to a CSV file
results_df = pd.DataFrame(results)
save_csv_file(results_df, f"{output_dir}/random_forest/data/results_after_removal.csv")
    

After Removal

In [ ]:
for i in range(n_splits):
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_train.csv")
    label_issue_index = pd.read_csv(f"{output_dir}/random_forest/data/label_issues.csv")
    train_df_removed = train_df[~train_df["Unnamed: 0"].isin(label_issue_index["Unnamed: 0"])]
    print(len(train_df_removed), len(train_df_removed)/len(train_df))
    save_csv_file(train_df_removed, f"{output_dir}/split_{i+1}/train_removed.csv")

In [ ]:
for i in range(n_splits):
    command = f"""
    python /kaggle/working/scarf/get_processed_dataset.py \
      --config_dir "configs" \
      --output_dir "{output_dir}/split_{i+1}" \
      --data_path "{output_dir}/split_{i+1}/train_removed.csv" \
      --output_csv_name "processed_train_removed" \
    """
    os.system(command)

In [ ]:
results = []

for i in range(n_splits):
    clf = get_rf()
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_train_removed.csv")
    test_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_test.csv")
    clf.fit(train_df.drop(columns=["EnergyRating"]), train_df["EnergyRating"])
    acc = clf.score(test_df.drop(columns=["EnergyRating"]), test_df["EnergyRating"])
    predict = clf.predict(test_df.drop(columns=["EnergyRating"]))
    f1 = f1_score(test_df["EnergyRating"], predict, average='macro')
    print(f"Split: {i+1}, Accuracy: {acc}, F1 Score: {f1}")
    results.append({"Split": i+1, "Accuracy": acc, "F1 Score": f1})

# Save the results to a CSV file
results_df = pd.DataFrame(results)
save_csv_file(results_df, f"{output_dir}/random_forest/data/results_after_removal.csv")

**Scarf Embedding**

In [ ]:

X_processed = pd.DataFrame(all_emb, columns=[f"feature_{i+1}" for i in range(all_emb.shape[1])])
labels = processed['EnergyRating']
clf = get_rf()
pred_probs = cross_val_predict(
    clf,
    X_processed,
    labels,
    cv=kf,
    method="predict_proba",
)
KNN = NearestNeighbors(metric='euclidean')
KNN.fit(X_processed.values)

knn_graph = KNN.kneighbors_graph(mode="distance")
data = {"X": X_processed.values, "y": labels}

lab = Datalab(data, label_name="y")
lab.find_issues(pred_probs=pred_probs, knn_graph=knn_graph)
display(lab.report())

In [ ]:
# Label Issues
label_issues_num =  52164
outlier_nums = 3752
duplicates_nums = 7983

issue_results = lab.get_issues("label")
sorted_issues = issue_results.sort_values("label_score").index

sorted_issues_df = pd.DataFrame({
    "Unnamed: 0": X_raw.loc[sorted_issues, "Unnamed: 0"]
})
save_csv_file(sorted_issues_df[:label_issues_num], f"{output_dir}/random_forest/emb/label_issues.csv")
print(len(sorted_issues_df))

print("Label Issues")
display(X_raw.iloc[sorted_issues].assign(
    given_label=labels.iloc[sorted_issues],
    predicted_label=issue_results["predicted_label"].iloc[sorted_issues]
).head())

# Outliers
outlier_results = lab.get_issues("outlier")
sorted_outliers= outlier_results.sort_values("outlier_score").index
print("Outliers")
display(X_raw.iloc[sorted_outliers].head())

sorted_outliers_df = pd.DataFrame({
    "Unnamed: 0": X_raw.loc[sorted_outliers[:outlier_nums], "Unnamed: 0"]
})
print(len(sorted_outliers_df))
save_csv_file(sorted_outliers_df, f"{output_dir}/random_forest/emb/outliers.csv")


# Near-duplicate issues
duplicate_results = lab.get_issues("near_duplicate")
sorted_duplicates = duplicate_results.sort_values("near_duplicate_score").index
print("Near-duplicate issues")
display(duplicate_results.sort_values("near_duplicate_score").head())
sorted_duplicates_df = pd.DataFrame({
    "Unnamed: 0": X_raw.loc[sorted_duplicates, "Unnamed: 0"]
})
print(len(sorted_duplicates_df))
save_csv_file(sorted_duplicates_df[:duplicates_nums], f"{output_dir}/random_forest/emb/duplicates.csv")


**Before Removal**

In [ ]:
results = []

for i in range(n_splits):
    clf = get_rf()
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_train.csv")
    test_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_test.csv")
    train_X = np.load(f"{output_dir}/split_{i+1}/train.npy")
    test_X = np.load(f"{output_dir}/split_{i+1}/test.npy")
    clf.fit(train_X, train_df["EnergyRating"])
    acc = clf.score(test_X, test_df["EnergyRating"],
             )
    predict = clf.predict(test_X)
    f1 = f1_score(test_df["EnergyRating"], predict, average='macro')
    print(f"Split: {i+1}, Accuracy: {acc}, F1 Score: {f1}")
    results.append({"Split": i+1, "Accuracy": acc, "F1 Score": f1})

# Save the results to a CSV file
results_df = pd.DataFrame(results)
save_csv_file(results_df, f"{output_dir}/random_forest/emb/results_before_removal.csv")

**After Removal**

In [ ]:
original_rating_encoding = {
    "A1": 0,
    "A2": 1,
    "A3": 2,
    "B1": 3,
    "B2": 4,
    "B3": 5,
    "C1": 6,
    "C2": 7,
    "C3": 8,
    "D1": 9,
    "D2": 10,
    "E1": 11,
    "E2": 12,
    "F": 13,
    "G": 14
}

In [ ]:
for i in range(n_splits):
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_train.csv")
    train_X = np.load(f"{output_dir}/split_{i+1}/train.npy")
    
    label_issue_index = pd.read_csv(f"{output_dir}/random_forest/emb/label_issues.csv")
    train_df_removed = train_df[~train_df["Unnamed: 0"].isin(label_issue_index["Unnamed: 0"])]
    train_X_removed = train_X[train_df_removed.index]
    train_df_removed['EnergyRating'] = train_df_removed['EnergyRating'].str.strip()
    train_df_removed['EnergyRating'] = train_df_removed['EnergyRating'].map(original_rating_encoding)
    print(len(train_df_removed), len(train_df_removed)/len(train_df))
    save_csv_file(train_df_removed, f"{output_dir}/split_{i+1}/train_removed.csv")
    np.save(f"{output_dir}/split_{i+1}/train_removed.npy", train_X_removed)

In [ ]:
results = []

for i in range(n_splits):
    clf = get_rf()
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/train_removed.csv")
    test_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_test.csv")
    train_X = np.load(f"{output_dir}/split_{i+1}//train_removed.npy")
    test_X = np.load(f"{output_dir}/split_{i+1}/test.npy")
    clf.fit(train_X, train_df["EnergyRating"])
    acc = clf.score(test_X, test_df["EnergyRating"],
             )
    predict = clf.predict(test_X)
    f1 = f1_score(test_df["EnergyRating"], predict, average='macro')
    print(f"Split: {i+1}, Accuracy: {acc}, F1 Score: {f1}")
    results.append({"Split": i+1, "Accuracy": acc, "F1 Score": f1})
results_df = pd.DataFrame(results)
save_csv_file(results_df, f"{output_dir}/random_forest/emb/results_after_removal.csv")



**MLP**

In [ ]:
from sklearn.neural_network import MLPClassifier
hidden_layer_sizes = [256 , 128 , 64 , 32 , 16]
max_iter = 25
batch_size = 32
def get_mlp(hidden_layer_sizes=hidden_layer_sizes,
           max_iter= max_iter,
           batch_size = batch_size):
    return MLPClassifier(hidden_layer_sizes=hidden_layer_sizes,
                        random_state= 42,
                        max_iter= max_iter,
                        batch_size = batch_size)

In [ ]:
X_processed = processed.drop(columns=["EnergyRating"])
labels = processed['EnergyRating']
clf = get_mlp()
pred_probs = cross_val_predict(
    clf,
    X_processed,
    labels,
    cv=kf,
    method="predict_proba",
)
KNN = NearestNeighbors(metric='euclidean')
KNN.fit(X_processed.values)

knn_graph = KNN.kneighbors_graph(mode="distance")
data = {"X": X_processed.values, "y": labels}

lab = Datalab(data, label_name="y")
lab.find_issues(pred_probs=pred_probs, knn_graph=knn_graph)

display(lab.report())

**Original Data**

In [ ]:
# Label Issues
label_issues_num =  36939
outlier_nums = 2296
duplicates_nums = 8785

issue_results = lab.get_issues("label")
sorted_issues = issue_results.sort_values("label_score").index

sorted_issues_df = pd.DataFrame({
    "Unnamed: 0": X_raw.loc[sorted_issues, "Unnamed: 0"]
})
save_csv_file(sorted_issues_df[:label_issues_num], f"{output_dir}/mlp/data/label_issues.csv")
print(len(sorted_issues_df))

print("Label Issues")
display(X_raw.iloc[sorted_issues].assign(
    given_label=labels.iloc[sorted_issues],
    predicted_label=issue_results["predicted_label"].iloc[sorted_issues]
).head())

# Outliers
outlier_results = lab.get_issues("outlier")
sorted_outliers= outlier_results.sort_values("outlier_score").index
print("Outliers")
display(X_raw.iloc[sorted_outliers].head())

sorted_outliers_df = pd.DataFrame({
    "Unnamed: 0": X_raw.loc[sorted_outliers[:outlier_nums], "Unnamed: 0"]
})
print(len(sorted_outliers_df))
save_csv_file(sorted_outliers_df, f"{output_dir}/mlp/data/outliers.csv")


# Near-duplicate issues
duplicate_results = lab.get_issues("near_duplicate")
sorted_duplicates = duplicate_results.sort_values("near_duplicate_score").index
print("Near-duplicate issues")
display(duplicate_results.sort_values("near_duplicate_score").head())
sorted_duplicates_df = pd.DataFrame({
    "Unnamed: 0": X_raw.loc[sorted_duplicates, "Unnamed: 0"]
})
print(len(sorted_duplicates_df))
save_csv_file(sorted_duplicates_df[:duplicates_nums], f"{output_dir}/mlp/data/duplicates.csv")


**Before Removal**

In [ ]:
results = []

for i in range(n_splits):
    clf = get_mlp()
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_train.csv")
    test_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_test.csv")
    clf.fit(train_df.drop(columns=["EnergyRating"]), train_df["EnergyRating"])
    acc = clf.score(test_df.drop(columns=["EnergyRating"]), test_df["EnergyRating"])
    predict = clf.predict(test_df.drop(columns=["EnergyRating"]))
    f1 = f1_score(test_df["EnergyRating"], predict, average='macro')
    print(f"Split: {i+1}, Accuracy: {acc}, F1 Score: {f1}")
    results.append({"Split": i+1, "Accuracy": acc, "F1 Score": f1})

# Save the results to a CSV file
results_df = pd.DataFrame(results)
save_csv_file(results_df, f"{output_dir}/mlp/data/results_before_removal.csv")

**After Removal**

In [ ]:
for i in range(n_splits):
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_train.csv")
    label_issue_index = pd.read_csv(f"{output_dir}/mlp/data/label_issues.csv")
    train_df_removed = train_df[~train_df["Unnamed: 0"].isin(label_issue_index["Unnamed: 0"])]
    print(len(train_df_removed), len(train_df_removed)/len(train_df))
    save_csv_file(train_df_removed, f"{output_dir}/split_{i+1}/train_removed.csv")

In [ ]:
for i in range(n_splits):
    command = f"""
    python /kaggle/working/scarf/get_processed_dataset.py \
      --config_dir "configs" \
      --output_dir "{output_dir}/split_{i+1}" \
      --data_path "{output_dir}/split_{i+1}/train_removed.csv" \
      --output_csv_name "processed_train_removed" \
    """
    os.system(command)

In [ ]:
results = []

for i in range(n_splits):
    clf = get_mlp()
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_train_removed.csv")
    test_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_test.csv")
    clf.fit(train_df.drop(columns=["EnergyRating"]), train_df["EnergyRating"])
    acc = clf.score(test_df.drop(columns=["EnergyRating"]), test_df["EnergyRating"])
    predict = clf.predict(test_df.drop(columns=["EnergyRating"]))
    f1 = f1_score(test_df["EnergyRating"], predict, average='macro')
    print(f"Split: {i+1}, Accuracy: {acc}, F1 Score: {f1}")
    results.append({"Split": i+1, "Accuracy": acc, "F1 Score": f1})

# Save the results to a CSV file
results_df = pd.DataFrame(results)
save_csv_file(results_df, f"{output_dir}/mlp/data/results_after_removal.csv")

**Scarf Embedding**

In [ ]:

X_processed = pd.DataFrame(all_emb, columns=[f"feature_{i+1}" for i in range(all_emb.shape[1])])
labels = processed['EnergyRating']
clf = get_mlp()
pred_probs = cross_val_predict(
    clf,
    X_processed,
    labels,
    cv=kf,
    method="predict_proba",
)
KNN = NearestNeighbors(metric='euclidean')
KNN.fit(X_processed.values)

knn_graph = KNN.kneighbors_graph(mode="distance")
data = {"X": X_processed.values, "y": labels}

lab = Datalab(data, label_name="y")
lab.find_issues(pred_probs=pred_probs, knn_graph=knn_graph)
display(lab.report())

In [ ]:
# Label Issues
label_issues_num =  48890
outlier_nums =  3752
duplicates_nums = 7983

issue_results = lab.get_issues("label")
sorted_issues = issue_results.sort_values("label_score").index

sorted_issues_df = pd.DataFrame({
    "Unnamed: 0": X_raw.loc[sorted_issues, "Unnamed: 0"]
})
save_csv_file(sorted_issues_df[:label_issues_num], f"{output_dir}/mlp/emb/label_issues.csv")
print(len(sorted_issues_df))

print("Label Issues")
display(X_raw.iloc[sorted_issues].assign(
    given_label=labels.iloc[sorted_issues],
    predicted_label=issue_results["predicted_label"].iloc[sorted_issues]
).head())

# Outliers
outlier_results = lab.get_issues("outlier")
sorted_outliers= outlier_results.sort_values("outlier_score").index
print("Outliers")
display(X_raw.iloc[sorted_outliers].head())

sorted_outliers_df = pd.DataFrame({
    "Unnamed: 0": X_raw.loc[sorted_outliers[:outlier_nums], "Unnamed: 0"]
})
print(len(sorted_outliers_df))
save_csv_file(sorted_outliers_df, f"{output_dir}/mlp/emb/outliers.csv")


# Near-duplicate issues
duplicate_results = lab.get_issues("near_duplicate")
sorted_duplicates = duplicate_results.sort_values("near_duplicate_score").index
print("Near-duplicate issues")
display(duplicate_results.sort_values("near_duplicate_score").head())
sorted_duplicates_df = pd.DataFrame({
    "Unnamed: 0": X_raw.loc[sorted_duplicates, "Unnamed: 0"]
})
print(len(sorted_duplicates_df))
save_csv_file(sorted_duplicates_df[:duplicates_nums], f"{output_dir}/mlp/emb/duplicates.csv")


**Before Removal**

In [ ]:
results = []

for i in range(n_splits):
    clf = get_mlp()
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_train.csv")
    test_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_test.csv")
    train_X = np.load(f"{output_dir}/split_{i+1}/train.npy")
    test_X = np.load(f"{output_dir}/split_{i+1}/test.npy")
    clf.fit(train_X, train_df["EnergyRating"])
    acc = clf.score(test_X, test_df["EnergyRating"],
             )
    predict = clf.predict(test_X)
    f1 = f1_score(test_df["EnergyRating"], predict, average='macro')
    print(f"Split: {i+1}, Accuracy: {acc}, F1 Score: {f1}")
    results.append({"Split": i+1, "Accuracy": acc, "F1 Score": f1})

# Save the results to a CSV file
results_df = pd.DataFrame(results)
save_csv_file(results_df, f"{output_dir}/mlp/emb/results_before_removal.csv")

**After Removal**

In [ ]:
for i in range(n_splits):
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_train.csv")
    train_X = np.load(f"{output_dir}/split_{i+1}/train.npy")
    
    label_issue_index = pd.read_csv(f"{output_dir}/mlp/emb/label_issues.csv")
    train_df_removed = train_df[~train_df["Unnamed: 0"].isin(label_issue_index["Unnamed: 0"])]
    train_X_removed = train_X[train_df_removed.index]
    train_df_removed['EnergyRating'] = train_df_removed['EnergyRating'].str.strip()
    train_df_removed['EnergyRating'] = train_df_removed['EnergyRating'].map(original_rating_encoding)
    print(len(train_df_removed), len(train_df_removed)/len(train_df))
    save_csv_file(train_df_removed, f"{output_dir}/split_{i+1}/train_removed.csv")
    np.save(f"{output_dir}/split_{i+1}/train_removed.npy", train_X_removed)

In [ ]:
results = []

for i in range(n_splits):
    clf = get_mlp()
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/train_removed.csv")
    test_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_test.csv")
    train_X = np.load(f"{output_dir}/split_{i+1}//train_removed.npy")
    test_X = np.load(f"{output_dir}/split_{i+1}/test.npy")
    clf.fit(train_X, train_df["EnergyRating"])
    acc = clf.score(test_X, test_df["EnergyRating"],
             )
    predict = clf.predict(test_X)
    f1 = f1_score(test_df["EnergyRating"], predict, average='macro')
    print(f"Split: {i+1}, Accuracy: {acc}, F1 Score: {f1}")
    results.append({"Split": i+1, "Accuracy": acc, "F1 Score": f1})
results_df = pd.DataFrame(results)
save_csv_file(results_df, f"{output_dir}/mlp/emb/results_after_removal.csv")



In [ ]:
import matplotlib.pyplot as plt
def compare_results(data_before, data_after, emb_before, emb_after, title):
    fig, ax = plt.subplots(2,1, figsize=(8, 8), sharex=True)
    fig.suptitle(title)

    # Plot Accuracy comparison
    bar_width = 0.2
    splits = data_before['Split']

    ax[0].bar(splits - 1.5 * bar_width, data_before['Accuracy'], width=bar_width, label='Data Before Removal')
    ax[0].bar(splits - 0.5 * bar_width, data_after['Accuracy'], width=bar_width, label='Data After Removal')
    ax[0].bar(splits + 0.5 * bar_width, emb_before['Accuracy'], width=bar_width, label='Emb Before Removal')
    ax[0].bar(splits + 1.5 * bar_width, emb_after['Accuracy'], width=bar_width, label='Emb After Removal')
    ax[0].set_title('Accuracy Comparison')
    ax[0].set_ylabel('Accuracy')

    # Plot F1 Score comparison
    ax[1].bar(splits - 1.5 * bar_width, data_before['F1 Score'], width=bar_width, label='Data Before Removal')
    ax[1].bar(splits - 0.5 * bar_width, data_after['F1 Score'], width=bar_width, label='Data After Removal')
    ax[1].bar(splits + 0.5 * bar_width, emb_before['F1 Score'], width=bar_width, label='Emb Before Removal')
    ax[1].bar(splits + 1.5 * bar_width, emb_after['F1 Score'], width=bar_width, label='Emb After Removal')
    ax[1].set_title('F1 Score Comparison')
    ax[1].set_xlabel('Split')
    ax[1].set_ylabel('F1 Score')

    # plt.subplots_adjust(right=0.8)

    # Create a single legend outside of the plot, on the right
    handles, labels = ax[0].get_legend_handles_labels()
    # fig.legend(handles, labels)
    fig.legend(handles, labels, loc='center left', bbox_to_anchor=(1, 0.5))

    # Display the plot
    plt.tight_layout(rect=[0, 0, 0.85, 0.95])
    plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

data_before = pd.read_csv(f"{output_dir}/random_forest/data/results_before_removal.csv")
data_after = pd.read_csv(f"{output_dir}/random_forest/data/results_after_removal.csv")
emb_before = pd.read_csv(f"{output_dir}/random_forest/emb/results_before_removal.csv")
emb_after = pd.read_csv(f"{output_dir}/random_forest/emb/results_after_removal.csv")

compare_results(data_before, data_after, emb_before, emb_after, "Random Forest")

data_before = pd.read_csv(f"{output_dir}/mlp/data/results_before_removal.csv")
data_after = pd.read_csv(f"{output_dir}/mlp/data/results_after_removal.csv")
emb_before = pd.read_csv(f"{output_dir}/mlp/emb/results_before_removal.csv")
emb_after = pd.read_csv(f"{output_dir}/mlp/emb/results_after_removal.csv")

compare_results(data_before, data_after, emb_before, emb_after, "MLP")


